In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
articles = pd.read_parquet('../input/hm-fashion-recommendation-parquet/articles.parquet')
sales = pd.read_parquet('../input/hm-fashion-recommendation-parquet/sales.parquet')
customers = pd.read_parquet('../input/hm-fashion-recommendation-parquet/customers.parquet')

This notebook explores how we can easily narrow down the search for the relevant articles for a week.

# Product launches

There are 105.542 articles in the data. However, not all of them are sold every week. There are about 150-400 new product launches every week.

In [ ]:
first_product_sales = sales.merge(articles).groupby('product_code', as_index=False).agg(first_sale=('week', 'min'))

In [ ]:
# first 5 weeks are not representative
px.bar(first_product_sales[first_product_sales.first_sale>5].groupby('first_sale', as_index=False).size(), x='first_sale', y='size', title='New Products per week')

Every week we have about 400-1.000 new articles of which about 50% are new product launches and 50% are new colors/prints/patterns for existing products.

In [ ]:
first_article_sales = sales.merge(articles).groupby(['article_id', 'product_code'], as_index=False).agg(first_sale=('week', 'min')).merge(first_product_sales.rename(columns={'first_sale': 'first_product_sale'}))

In [ ]:
first_article_sales['new_product']=first_article_sales.first_sale==first_article_sales.first_product_sale

In [ ]:
# first 5 weeks are not representative
px.bar(first_article_sales[first_article_sales.first_sale>5].groupby(['first_sale', 'new_product'], as_index=False).size(), x='first_sale', y='size', color='new_product', title='New Articles per week')

This matches quite nicely the 995 new articles without sales transactions in the data.

In [ ]:
outer_join = articles.merge(first_article_sales, how = 'outer', indicator = True)
new_articles_without_transactions = outer_join[~(outer_join._merge == 'both')].drop('_merge', axis = 1)
len(new_articles_without_transactions)

# Relevant articles per week

About of the 14.000-25.000 articles of the 105.542 are sold every week. However, it's not obvious which ones have been discontinued or are currently out of stock to narrow down the search.

In [ ]:
px.bar(sales.groupby('week', as_index=False).agg(unique_articles=('article_id', pd.Series.nunique)), x='week', y='unique_articles', title='Number of unique articles per week')

Most likely the articles that are sold in one week have been sold in the week before. Let's check that distribution.

In [ ]:
sales_per_week = sales.groupby(['article_id', 'week'], as_index=False).agg(unit_sales=('price', 'size')).sort_values('week')
sales_per_week['last_purchase_week'] = sales_per_week.groupby('article_id').week.diff()
sales_per_week.loc[sales_per_week.last_purchase_week.isna(), 'last_purchase_week'] = 0
sales_per_week['last_purchase_week_bin'] = pd.cut(sales_per_week.last_purchase_week, bins=[0, 1, 2, 3, 4, 5, 105], labels=['new', '1 week before', '2 weeks before', '3 weeks before', '4 weeks before', '>=5 weeks before'], right=False)

In [ ]:
sales_per_week = sales_per_week.groupby(['week', 'last_purchase_week_bin']).agg(unit_sales=('unit_sales', 'sum'), article_count=('article_id', pd.Series.nunique)).reset_index()
sales_per_week['unit_sales_pct'] = sales_per_week['unit_sales']/sales_per_week.groupby('week').unit_sales.transform('sum')*100

Indeed most of the articles have been sold in the week before.

In [ ]:
px.bar(sales_per_week, x='week', y='article_count', color='last_purchase_week_bin', title='Number of unique articles per week')

The spike in articles sold in week 85 that had not been sold in the previous 4 weeks are likely "offline articles" that could not be sold due to the COVID19 lockdown and are not available in the online store.

Looking at the actual sales figures, the articles that were last sold a few weeks ago become even more irrelevant.

In [ ]:
px.bar(sales_per_week, x='week', y='unit_sales', color='last_purchase_week_bin', title='Unit sales per week grouped by week of last purchase')

In fact, this week's new articles and those sold in the previous three weeks already account for more than 99% of all sales. Normally it should be enough to look at only those 20k articles instead of all 105k articles.

In [ ]:
tmp = (sales_per_week[sales_per_week.week>5].groupby('last_purchase_week_bin').mean()).reset_index()
tmp['unit_sales_pct_cum'] = tmp['unit_sales_pct'].cumsum()
tmp['article_count_cum'] = tmp['article_count'].cumsum().astype('int')
tmp[['last_purchase_week_bin', 'unit_sales_pct', 'unit_sales_pct_cum', 'article_count_cum']]

In [ ]:
px.bar(sales_per_week, x='week', y='unit_sales_pct', color='last_purchase_week_bin', title='Relative sales grouped by week of last purchase')

# Looking at new customers

Reducing the number of relevant customers is a little harder, as the number of customers that purchase today and haven't purchased anything in the last 4 weeks is significant and there are a lot of customers in the data.

In [ ]:
len(customers)

In [ ]:
sales_per_week = sales.groupby(['customer_id', 'week'], as_index=False).agg(unit_sales=('price', 'size')).sort_values('week')
sales_per_week['last_purchase_week'] = sales_per_week.groupby('customer_id').week.diff()
sales_per_week.loc[sales_per_week.last_purchase_week.isna(), 'last_purchase_week'] = 0

In [ ]:
sales_per_week['last_purchase_week_bin'] = pd.cut(
    sales_per_week.last_purchase_week,
    bins=[0,
          1, 2,
          4, 9, 17, 26, 52,
          205],
    labels=['new',
            '1 week before', '2 weeks before',
            '1 month before', '2-3 months before', '4-6 months before', '6-12 months before',
            '>=1 year before'], right=False)

In [ ]:
sales_per_week_plot = sales_per_week.groupby(['week', 'last_purchase_week_bin']).agg(unit_sales=('unit_sales', 'sum'), customer_count=('customer_id', pd.Series.nunique)).reset_index()
sales_per_week_plot['unit_sales_pct'] = sales_per_week_plot['unit_sales']/sales_per_week_plot.groupby('week').unit_sales.transform('sum')*100

In [ ]:
px.bar(sales_per_week_plot, x='week', y='customer_count', color='last_purchase_week_bin', title='Number of unique customers per week')

In [ ]:
tmp = (sales_per_week_plot[sales_per_week_plot.week>40].groupby('last_purchase_week_bin').mean()).reset_index()
tmp['unit_sales_pct_cum'] = tmp['unit_sales_pct'].cumsum()
tmp['customer_count_cum'] = tmp['customer_count'].cumsum().astype('int')
tmp[['last_purchase_week_bin', 'unit_sales_pct', 'unit_sales_pct_cum', 'customer_count_cum']]

Focussing on customers that bought something in the last month would account for 70% of the sold items and cut the number of relevant customers to about 20%.

In [ ]:
one_months_customers = sales[sales.week.between(104-4, 104)].customer_id.nunique()
print(f'Number of customers in a month: {one_months_customers}')
print(f'Relative number of customers in a month: {one_months_customers/len(customers):.2%}')

The number of new customers seems reasonable in line with the transaction data.

In [ ]:
outer_join = customers.merge(sales, how = 'outer', indicator = True)
new_customers_without_transactions = outer_join[~(outer_join._merge == 'both')].drop('_merge', axis = 1)
len(new_customers_without_transactions)